In [56]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression, LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

VITALS = ['LABEL_RRate', 'LABEL_ABPm', 'LABEL_SpO2', 'LABEL_Heartrate']
TESTS = ['LABEL_BaseExcess', 'LABEL_Fibrinogen', 'LABEL_AST', 'LABEL_Alkalinephos', 'LABEL_Bilirubin_total',
         'LABEL_Lactate', 'LABEL_TroponinI', 'LABEL_SaO2',
         'LABEL_Bilirubin_direct', 'LABEL_EtCO2']

In [57]:
train_features = pd.read_csv('train_features.csv')
test_features = pd.read_csv('test_features.csv')

train_labels = pd.read_csv('train_labels.csv').sort_values(by='pid')

pids = test_features['pid'].drop_duplicates().sort_values().reset_index(drop=True)

train_features

,pid,Time,Age,EtCO2,PTT,BUN,Lactate,Temp,Hgb,HCO3,...,Alkalinephos,SpO2,Bilirubin_direct,Chloride,Hct,Heartrate,Bilirubin_total,TroponinI,ABPs,pH
0,1,3,34.0,NaN,NaN,12.0,NaN,36.0,8.7,24.0,...,NaN,100.0,NaN,114.0,24.6,94.0,NaN,NaN,142.0,7.33
1,1,4,34.0,NaN,NaN,NaN,NaN,36.0,NaN,NaN,...,NaN,100.0,NaN,NaN,NaN,99.0,NaN,NaN,125.0,7.33
2,1,5,34.0,NaN,NaN,NaN,NaN,36.0,NaN,NaN,...,NaN,100.0,NaN,NaN,NaN,92.0,NaN,NaN,110.0,7.37
3,1,6,34.0,NaN,NaN,NaN,NaN,37.0,NaN,NaN,...,NaN,100.0,NaN,NaN,NaN,88.0,NaN,NaN,104.0,7.37
4,1,7,34.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,100.0,NaN,NaN,22.4,81.0,NaN,NaN,100.0,7.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227935,9999,8,85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,80.0,NaN,NaN,110.0,NaN
227936,9999,9,85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,83.0,NaN,NaN,123.0,NaN
227937,9999,10,85.0,NaN,NaN,NaN,NaN,36.0,NaN,NaN,...,NaN,98.0,NaN,NaN,NaN,80.0,NaN,NaN,138.0,NaN
227938,9999,11,85.0,NaN,NaN,NaN,NaN,NaN,10.2,NaN,...,NaN,98.0,NaN,NaN,31.0,75.0,NaN,NaN,125.0,NaN


In [45]:
def feature_engineer(data):
    data = (data.groupby('pid').mean()).fillna(data.median())
    data = data.drop('Time', axis=1).sort_values(by='pid')

    scaler = StandardScaler()
    return scaler.fit_transform(data)


In [46]:
X = feature_engineer(train_features)

X_submission = feature_engineer(test_features)

In [47]:
scores = []
y_pred = []

for test in TESTS + ['LABEL_Sepsis']:
    y = train_labels[test]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    model = LogisticRegression(random_state=32).fit(X_train, y_train)
    scores.append(model.score(X_test, y_test))

    y_pred.append(model.predict_proba(X_submission)[:,1])

scores

[0.7627653974381471,
 0.9324442884716617,
 0.7868047025793999,
 0.7920687839971925,
 0.7887348657659239,
 0.8164590278996315,
 0.9103351465169328,
 0.8029478855939639,
 0.9666608176873136,
 0.9340235128969995,
 0.9426215125460607]

In [48]:
labels_tests = pd.DataFrame(np.transpose(y_pred), columns=(TESTS + ['LABEL_Sepsis']))
labels_tests.head(20)

,LABEL_BaseExcess,LABEL_Fibrinogen,LABEL_AST,LABEL_Alkalinephos,LABEL_Bilirubin_total,LABEL_Lactate,LABEL_TroponinI,LABEL_SaO2,LABEL_Bilirubin_direct,LABEL_EtCO2,LABEL_Sepsis
0,0.430861,0.526732,0.999519,0.999433,0.999490,0.477557,0.008800,0.281171,0.127607,0.211423,0.134063
1,0.120834,0.055385,0.308823,0.309664,0.316681,0.132354,0.183899,0.196462,0.029559,0.037826,0.030964
2,0.252097,0.047740,0.174448,0.171533,0.177333,0.157479,0.064857,0.209998,0.023352,0.038265,0.051240
3,0.915501,0.536227,0.990402,0.989842,0.991532,0.517301,0.009557,0.900630,0.150142,0.000014,0.051194
4,0.211334,0.027014,0.358793,0.357392,0.367929,0.130878,0.150482,0.164371,0.024251,0.024981,0.080673
5,0.074344,0.021954,0.197548,0.194029,0.192804,0.069889,0.169441,0.091993,0.018711,0.043749,0.031822
6,0.031490,0.052753,0.345306,0.339252,0.341091,0.348342,0.244687,0.442916,0.058988,0.233205,0.059942
7,0.633419,0.156491,0.042263,0.039758,0.037145,0.375436,0.046826,0.725814,0.025754,0.015169,0.046069
8,0.222891,0.050431,0.055883,0.051884,0.045562,0.157093,0.097805,0.220345,0.021860,0.040236,0.047755
9,0.483607,0.030227,0.419012,0.416288,0.455640,0.138720,0.142036,0.182820,0.033683,0.010799,0.153850


In [49]:
scores = []
y_pred = []

for vital in VITALS:
    y = train_labels[vital]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    model = LassoCV(random_state=32).fit(X_train, y_train)
    scores.append(model.score(X_test, y_test))
    y_pred.append(model.predict(X_submission))

scores

[0.39493444559915114,
 0.6010213826254593,
 0.3619866585828676,
 0.6120118970537956]

In [50]:
y_pred

[array([14.76442732, 17.93430219, 18.78633528, ..., 18.01603238,
        19.56639414, 18.13346382]),
 array([81.52201788, 85.73393602, 73.08058334, ..., 77.34402299,
        94.08561361, 84.35816898]),
 array([98.79399897, 96.50970159, 95.85093817, ..., 98.25844989,
        97.49659173, 98.6043507 ]),
 array([ 86.15570832,  96.80844278,  70.03263898, ...,  86.72350595,
        118.35054716, 107.8817294 ])]

In [51]:
labels_vitals = pd.DataFrame(np.transpose(y_pred), columns=VITALS)
labels_vitals.head()

,LABEL_RRate,LABEL_ABPm,LABEL_SpO2,LABEL_Heartrate
0,14.764427,81.522018,98.793999,86.155708
1,17.934302,85.733936,96.509702,96.808443
2,18.786335,73.080583,95.850938,70.032639
3,18.224721,85.094112,98.097519,93.821351
4,19.669912,88.520382,96.345225,89.402206


In [52]:
result = pd.concat([pids, labels_tests, labels_vitals], axis=1)

In [53]:
result.to_csv('prediction.zip', index=False, float_format='%.3f', compression='zip')

In [55]:
pred = pd.read_csv('sample.zip')
pred

,pid,LABEL_BaseExcess,LABEL_Fibrinogen,LABEL_AST,LABEL_Alkalinephos,LABEL_Bilirubin_total,LABEL_Lactate,LABEL_TroponinI,LABEL_SaO2,LABEL_Bilirubin_direct,LABEL_EtCO2,LABEL_Sepsis,LABEL_RRate,LABEL_ABPm,LABEL_SpO2,LABEL_Heartrate
0,0,0.940,0.341,0.597,0.651,0.557,0.745,0.224,0.363,0.506,0.643,0.162,18.796,82.511,96.947,84.12
1,10001,0.773,0.320,0.451,0.152,0.001,0.525,0.276,0.327,0.316,0.656,0.486,18.796,82.511,96.947,84.12
2,10003,0.741,0.211,0.348,0.153,0.859,0.446,0.406,0.607,0.757,0.290,0.451,18.796,82.511,96.947,84.12
3,10004,0.147,0.312,0.733,0.129,0.356,0.367,0.931,0.715,0.434,0.005,0.785,18.796,82.511,96.947,84.12
4,10005,0.255,0.746,0.587,0.743,0.248,0.330,0.071,0.291,0.399,0.217,0.040,18.796,82.511,96.947,84.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12659,9989,0.943,0.541,0.373,0.944,0.562,0.594,0.838,0.938,0.401,0.195,0.647,18.796,82.511,96.947,84.12
12660,9991,0.561,0.040,0.095,0.667,0.918,0.323,0.784,0.343,0.552,0.047,0.916,18.796,82.511,96.947,84.12
12661,9992,0.112,0.962,0.967,0.564,0.064,0.545,0.210,0.853,0.429,0.829,0.093,18.796,82.511,96.947,84.12
12662,9994,0.892,0.540,0.868,0.201,0.259,0.632,0.282,0.810,0.724,0.074,0.936,18.796,82.511,96.947,84.12
